In [ ]:
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O 'Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1qhb0j5YJwlhjBRU5X9ttsxOegkD5bzOv' -O 'input_video.mp4'

!pip install -q youtube-dl
!pip install ffmpeg-python


!pip install flask
!pip install pyngrok


!pip install flask-ngrok
!pip install flask-bootstrap
!pip install flask_cors


!pip install tensorflow==2.9.0
!pip install numpy==1.20


!pip install TTS

!ngrok authtoken 2Nn103R7KQ7Qqd9PfflgCzUspha_cvMpzbYnmrbLbGCksTMD

!pip install transformers

!pip install librosa==0.8.0


Cloning into 'Wav2Lip'...
remote: Enumerating objects: 378, done.
remote: Total 378 (delta 0), reused 0 (delta 0), pack-reused 378
Receiving objects: 100% (378/378), 526.97 KiB | 837.00 KiB/s, done.
Resolving deltas: 100% (205/205), done.
--2023-05-05 09:40:42--  https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA
Resolving iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)... 13.107.136.8, 13.107.138.8, 2620:1ec:8f8::8, ...
Connecting to iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)|13.107.136.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 435801865 (416M) [application/octet-stream]
Saving to: ‘Wav2Lip/checkpoints/wav2lip_gan.pth’

Wav2Lip/checkpoints 100%[===================>] 415.61M  70.2MB/s    in 9.9s    

2023-05-05 09:40:53 (41.8 MB/s) - ‘Wav2Lip/checkpoints/wav2lip_gan.pth’ saved [435801865/435801865]

Looking in indexes: https:

In [11]:
def runcmd(cmd, verbose = False, *args, **kwargs):
  process = subprocess.Popen(
      cmd,
      stdout = subprocess.PIPE,
      stderr = subprocess.PIPE,
      text = True,
      shell = True
  )
  std_out, std_err = process.communicate()
  if verbose:
      print(std_out.strip(), std_err)
  pass

def generate_audio(text):
  audio_file = time.strftime("%Y%m%d-%H%M%S")+'.wav'
  tts.tts_to_file(text=text, speaker=tts.speakers[4], language=tts.languages[0], file_path=audio_file)
  return('/content/'+audio_file)

def generate_video(audio_path, video_path):
  runcmd(f"cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face {video_path} --audio {audio_path} --resize_factor 2", verbose=True)
  return('/content/Wav2Lip/results/result_voice.mp4')

def clean_text(text):
  # Remove website links
  text = re.sub(r'http\S+', '', text)
  # Remove unnecessary punctuations
  text = re.sub(r'[^\w\s,\.]', '\n', text)
  # Remove extra whitespaces
  text = re.sub(r'\s+', ' ', text)
  # Remove periods not preceded by any character
  text = re.sub(r'(?<!\w)\.', '', text)
  # Replace multiple white spaces with a single white space
  text = re.sub(r'\s+', ' ', text)
  return text.strip()

def generate_article(query):
  startTime = time.time()
  input_ids = tokenizer.encode(query, return_tensors='tf')
  tf.random.set_seed(0)

  sample_outputs = model.generate(
      input_ids,
      do_sample=True, 
      max_length=400, 
      top_k=50,
      top_p=0.95, 
      num_return_sequences=2
  )
  text = ''
  for i, sample_output in enumerate(sample_outputs):
      text += f'{tokenizer.decode(sample_output, skip_special_tokens=True)}'.replace(query,'')
      text += '. '
  executionTime = (time.time() - startTime)
  return clean_text(text), executionTime

In [ ]:
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
from TTS.api import TTS
import time
import subprocess
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import re


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

model_name = TTS.list_models()[0]

tts = TTS(model_name)

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)


@app.route('/')
def home():
    return '<h1>Hello, World!</h1>'

@app.route('/generate_audio', methods=['POST'])
def generate_audio_route():
    text = request.form['text']
    audio_path = generate_audio(text)
    return send_file(
         audio_path, 
         mimetype="audio/wav",
         as_attachment=True)


@app.route('/generate_video', methods=['POST'])
def generate_video_route():
    text = request.form['text']
    audio_path = generate_audio(text)
    video_path = '/content/input_video.mp4'
    output_path = generate_video(audio_path, video_path)
    return send_file(
         output_path, 
         mimetype="video/mp4", 
         as_attachment=True)


@app.route('/generate_article', methods=['POST'])
def generate_article_route():
    query = request.form['query']
    article, timetaken = generate_article(query)
    if article.strip() == '':
        article = 'Unable to generate the article for the given query, Please try with another query'
    return jsonify({"article": article, "time_taken":timetaken})

app.run()

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


No API token found for 🐸Coqui Studio voices - https://coqui.ai 
Visit 🔗https://app.coqui.ai/account to get one.
Set it as an environment variable `export COQUI_STUDIO_TOKEN=<token>`

 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://7acd-35-196-93-81.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [05/May/2023 10:23:07] "POST /generate_article HTTP/1.1" 200 -


 > Text splitted to sentences.
['to test OpenAI in mobile, business and financial services.', 'As an external mentor, Amit should have demonstrated his skills in the early stages of OpenAI s development, as well as other important issues like the ability to implement OpenAI into existing open technologies like SmartGlass, which will allow the development of open source, public, open software applications.', 'Vaid also helped develop the Open AI Collaboration project IAC This project, which aims to allow open access of ideas through distributed software, has been a key element for OpenAI s development.', 'One of the biggest obstacles to open source open source software, however, is the lack of a license to license OpenAI in the UK, to develop software and code.', 'If Vaid is unable to get a UK licence that permits open access of the open software to the open project, for example by a European Union government, the project could end up in the UK without the license.', 'In my interview wi

INFO:werkzeug:127.0.0.1 - - [05/May/2023 10:25:38] "POST /generate_video HTTP/1.1" 200 -


Using cuda for inference.
Reading video frames...
Number of frames available for inference: 515
(80, 7492)
Length of mel chunks: 2338
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded 
  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  3%|▎         | 1/33 [00:26<13:56, 26.15s/it]

  6%|▌         | 2/33 [00:27<06:03, 11.74s/it]

  9%|▉         | 3/33 [00:29<03:31,  7.07s/it]

 12%|█▏        | 4/33 [00:30<02:21,  4.87s/it]

 15%|█▌        | 5/33 [00:32<01:42,  3.66s/it]

 18%|█▊        | 6/33 [00:33<01:19,  2.95s/it]

 21%|██        | 7/33 [00:35<01:04,  2.49s/it]

 24%|██▍       | 8/33 [00:36<00:54,  2.19s/it]

 27%|██▋       | 9/33 [00:38<00:47,  2.00s/it]

 30%|███       | 10/33 [00:40<00:42,  1.85s/it]

 33%|███▎      | 11/33 [00:41<00:38,  1.76s/it]

 36%|███▋      | 12/33 [00:43<00:35,  1.69s/it]

 39%|███▉      | 13/33 [00:44<00:32,  1.64s/it]

 42%|████▏     | 14/33 [00:46<00:30,  1.61s/it]

 45%|████▌     | 15/33 [00:47<00:28,  1.60s/i

In [ ]:
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
from TTS.api import TTS
import time
import subprocess
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import re


#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

#model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

model_name = TTS.list_models()[0]

tts = TTS(model_name)




def runcmd(cmd, verbose = False, *args, **kwargs):
  process = subprocess.Popen(
      cmd,
      stdout = subprocess.PIPE,
      stderr = subprocess.PIPE,
      text = True,
      shell = True
  )
  std_out, std_err = process.communicate()
  if verbose:
      print(std_out.strip(), std_err)
  pass

def generate_audio(text):
  audio_file = time.strftime("%Y%m%d-%H%M%S")+'.wav'
  tts.tts_to_file(text=text, speaker=tts.speakers[4], language=tts.languages[0], file_path=audio_file)
  return('/content/'+audio_file)

def generate_video(audio_path, video_path):
  runcmd(f"cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face {video_path} --audio {audio_path} --resize_factor 2", verbose=True)
  return('/content/Wav2Lip/results/result_voice.mp4')

No API token found for 🐸Coqui Studio voices - https://coqui.ai 
Visit 🔗https://app.coqui.ai/account to get one.
Set it as an environment variable `export COQUI_STUDIO_TOKEN=<token>`

 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts


100%|██████████| 425M/425M [00:06<00:00, 65.6MiB/s]


 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | 

In [ ]:
text = "Good morning everyone,I'm Amit Vaid, the lead of AI India and working with Bionics team. Today, I'm excited to share with you some important updates about our progress with generative AI models, particularly ChatGPT, and the establishment of an internal OpenAI Innovation lab in StateStreet.As you all know, generative AI models like ChatGPT have been making significant breakthroughs in the field of AI, enabling machines to understand human language and even mimic human-like conversations. ChatGPT is one such model that has gained popularity for its ability to generate human-like text and engage in natural conversations,At StateStreet, we have been exploring the potential of generative AI models like ChatGPT to revolutionize the banking industry. With these models, we can create personalized chatbots that can assist customers with their queries and provide them with a seamless banking experience. We have already made significant progress in this area and are excited to explore further applications of these models.In addition to our work with generative AI models, I'm thrilled to announce that we are establishing an internal OpenAI Innovation lab here in StateStreet. This lab will serve as a hub for our AI research and development efforts, allowing us to collaborate with top AI experts and stay at the forefront of AI innovation.Our product portfolio will include the usage of open AI in the banking domain and inside StateStreet. We are committed to leveraging the power of AI to drive business growth, improve customer experience, and enhance the overall efficiency of our operations.I'm excited about the possibilities that lie ahead of us, and I look forward to working with each of you as we continue to push the boundaries of AI innovation. Thank you."

In [ ]:
generate_audio(text)

 > Text splitted to sentences.
["Good morning everyone,I'm Amit Vaid, the lead of AI India and working with Bionics team.", "Today, I'm excited to share with you some important updates about our progress with generative AI models, particularly ChatGPT, and the establishment of an internal OpenAI Innovation lab in StateStreet.", 'As you all know, generative AI models like ChatGPT have been making significant breakthroughs in the field of AI, enabling machines to understand human language and even mimic human-like conversations.', 'ChatGPT is one such model that has gained popularity for its ability to generate human-like text and engage in natural conversations,At StateStreet, we have been exploring the potential of generative AI models like ChatGPT to revolutionize the banking industry.', 'With these models, we can create personalized chatbots that can assist customers with their queries and provide them with a seamless banking experience.', 'We have already made significant progress i

'/content/20230505-094903.wav'

In [10]:
generate_video('/content/20230505-094903.wav', 'content/input_video.mp4')

Using cuda for inference. Traceback (most recent call last):
  File "/content/Wav2Lip/inference.py", line 280, in <module>
    main()
  File "/content/Wav2Lip/inference.py", line 183, in main
    raise ValueError('--face argument must be a valid path to video/image file')
ValueError: --face argument must be a valid path to video/image file



'/content/Wav2Lip/results/result_voice.mp4'